In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
import statistics
from scipy.stats import poisson,skellam

world_cup_matches = pd.read_csv("C:\\ic-futebol\\dataset\\results.csv")
world_cup_matches['year'] = world_cup_matches['date'].str[:4].astype(int)
world_cup_matches['month'] = world_cup_matches['date'].str[5:7].astype(int)
world_cup_matches['day'] = world_cup_matches['date'].str[8:].astype(int)
world_cup_matches = world_cup_matches[['home_team', 'home_score', 'away_score', 'away_team', 'tournament', 'year', 'month', 'day']]
world_cup_matches = world_cup_matches[(world_cup_matches.year) >= 2015]
world_cup_matches = world_cup_matches[(world_cup_matches.year) <= 2018]

teams = ['Russia', 'Brazil', 'Australia', 'Iran', 'Japan', 'Saudi Arabia', 'South Korea', 'Egypt', 'Morocco', 'Nigeria', 'Senegal', 'Tunisia', 'Costa Rica', 'Mexico', 'Panama', 'Argentina', 'Colombia', 'Peru', 'Uruguay', 'Belgium', 'Croatia', 'Denmark', 'England', 'France', 'Germany', 'Iceland', 'Poland', 'Portugal', 'Serbia', 'Spain', 'Sweden', 'Switzerland']
teams.sort()
world_cup_matches = world_cup_matches[world_cup_matches.home_team.isin(teams) | world_cup_matches.away_team.isin(teams)]

world_cup_matches.head()

,home_team,home_score,away_score,away_team,tournament,year,month,day
36610,Iran,1,0,Iraq,Friendly,2015,1,4
36611,South Korea,2,0,Saudi Arabia,Friendly,2015,1,4
36615,Australia,4,1,Kuwait,AFC Asian Cup,2015,1,9
36616,Senegal,1,0,Gabon,Friendly,2015,1,9
36620,South Korea,1,0,Oman,AFC Asian Cup,2015,1,10


In [6]:
world_cup_matches.describe()

,home_score,away_score,year,month,day
count,1295.000000,1295.000000,1295.000000,1295.000000,1295.000000
mean,1.562934,1.094981,2016.494208,6.925869,14.884170
std,1.521261,1.257840,1.106293,3.178473,8.641791
min,0.000000,0.000000,2015.000000,1.000000,1.000000
25%,0.000000,0.000000,2016.000000,5.000000,8.000000
50%,1.000000,1.000000,2016.000000,6.000000,13.000000
75%,2.000000,2.000000,2017.000000,10.000000,23.000000
max,10.000000,10.000000,2018.000000,12.000000,31.000000


In [8]:
teams2 = world_cup_matches['home_team'].unique().tolist()
teams3 = world_cup_matches['away_team'].unique().tolist()
teams = list(set(teams2).union(set(teams3)))
data = {'Team': teams, 'AttackForce': [0 for i in range(len(teams))], 'DefenseForce': [0 for i in range(len(teams))]}
stats = pd.DataFrame(data)
for time in teams:
    team_goals = 0.0
    conceded_goals = 0.0
    team_matches = 0.0
    time_model = world_cup_matches[(world_cup_matches['home_team'] == time) | (world_cup_matches['away_team'] == time)]
    #print(time_model)
    for index, row in time_model.iterrows():
        if row['home_team'] == time:
            team_goals += row['home_score'] 
            conceded_goals += row['away_score'] 

        if row['away_team'] == time:
            team_goals += row['away_score']
            conceded_goals += row['home_score']
            
        team_matches += 1
        
    stats.loc[stats['Team'] == time, 'AttackForce'] = team_goals/(team_matches)
    stats.loc[stats['Team'] == time, 'DefenseForce'] = (conceded_goals/(team_matches))
    
stats2 = stats.set_index('Team')

print(stats2)
world_cup_matches['HomeAttack'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['HomeDefence'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['AwayAttack'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['AwayDefence'] = [0.0 for i in range(len(world_cup_matches))]

for index, row in world_cup_matches.iterrows():
    world_cup_matches.loc[index,'HomeAttack'] = (stats2.loc[row['home_team'], 'AttackForce'] - statistics.mean(stats2['AttackForce'])) / statistics.stdev(stats2['AttackForce'])
    world_cup_matches.loc[index,'HomeDefence'] = (stats2.loc[row['home_team'], 'DefenseForce'] - statistics.mean(stats2['DefenseForce'])) / statistics.stdev(stats2['DefenseForce'])
    world_cup_matches.loc[index,'AwayAttack'] = (stats2.loc[row['away_team'], 'AttackForce']- statistics.mean(stats2['AttackForce'])) / statistics.stdev(stats2['AttackForce'])
    world_cup_matches.loc[index,'AwayDefence'] = (stats2.loc[row['away_team'], 'DefenseForce'] - statistics.mean(stats2['DefenseForce'])) / statistics.stdev(stats2['DefenseForce'])
world_cup_matches.head()

                       AttackForce  DefenseForce
Team                                            
Republic of Ireland       0.736842      1.315789
Bangladesh                0.000000      4.500000
Kosovo                    0.250000      2.750000
Egypt                     1.511111      0.755556
Bahrain                   0.000000      4.000000
Costa Rica                1.215686      1.352941
Rwanda                    0.333333      2.000000
Honduras                  0.588235      1.235294
Zambia                    0.500000      2.000000
Lebanon                   0.000000      2.333333
Burundi                   1.000000      3.000000
Italy                     0.880000      1.160000
Grenada                   0.000000      5.000000
Iceland                   1.490909      1.472727
Poland                    1.840909      1.113636
Guatemala                 0.250000      2.750000
Gabon                     0.600000      1.400000
Liechtenstein             0.000000      5.285714
Australia           

,home_team,home_score,away_score,away_team,year,month,day,HomeAttack,HomeDefence,AwayAttack,AwayDefence
36610,Iran,1,0,Iraq,2015,1,4,1.558757,-1.116774,0.001613,-0.369693
36611,South Korea,2,0,Saudi Arabia,2015,1,4,1.209519,-0.915634,1.460912,-0.595349
36615,Australia,4,1,Kuwait,2015,1,9,1.830550,-0.818112,-0.369675,0.089058
36616,Senegal,1,0,Gabon,2015,1,9,1.261361,-1.014963,-0.325670,-0.525916
36620,South Korea,1,0,Oman,2015,1,10,1.209519,-0.915634,-0.589697,0.188247


In [10]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# essa tabela duplica os jogos, para ter apenas uma variavel alvo: gols
# ela transforma as colunas em time, oponente, gols, fora o resto das variaveis
# ou seja, os gols sao sempre referentes ao "time", contra o "oponente"
# nao existe mais dentro e fora de casa, ja que a tabela foi duplicada
goal_model_data = pd.concat([world_cup_matches[['home_team','away_team', 'HomeAttack', 'HomeDefence', 'AwayAttack', 'AwayDefence', 'home_score']].rename(
            columns={'home_team':'team', 'away_team':'opponent', 'HomeAttack':'att', 'HomeDefence': 'defe', 'AwayAttack': 'away_att', 'AwayDefence': 'away_def', 'home_score':'goals'}),
           world_cup_matches[['away_team','home_team', 'AwayAttack', 'AwayDefence','HomeAttack', 'HomeDefence', 'away_score']].rename(
            columns={'away_team':'team', 'home_team':'opponent', 'AwayAttack':'att', 'AwayDefence': 'defe', 'HomeAttack': 'away_att', 'HomeDefence': 'away_def','away_score':'goals'})])

#print(goal_model_data)

# definicao dos parametros, tendo como variavel alvo os gols e o resto como entrada
poisson_model = smf.glm(formula="goals ~ att + defe + away_att + away_def", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                 2590
Model:                            GLM   Df Residuals:                     2585
Model Family:                 Poisson   Df Model:                            4
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3519.6
Date:                Wed, 09 Oct 2019   Deviance:                       2763.8
Time:                        14:32:49   Pearson chi2:                 2.41e+03
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0195      0.044     -0.438      0.661      -0.107       0.068
att            0.3579      0.027     13.070      0.000       0.304       0.412
defe          -0.2113      0.061     -3.474      0.001      -0.330      -0.092
away_att      -0.0452      0.023     -1.939      0.053      -0.091       0.000
away_def       0.3985      0.022     18.192      0.000       0.356       0.441
==============================================================================
"""

In [21]:
def get_team_stats(team, return_df=False):
    #print(stats2)
    attack = stats2.loc[team, 'AttackForce']
    defence = stats2.loc[team, 'DefenseForce']
    
    # teste para influencia da variavel year
    #year = world_cup_matches.loc[team, 'year']
    #print(attack, defence)
    return [attack, defence]

def simulate_goals(team_pred, game, a_team='A Team', b_team='B Team', verbose=False):
    #a_goals = np.argmax(team_pred[0])
    #b_goals = np.argmax(team_pred[1])
    
    #print(a_goals)
    #print(b_goals)
    a_goals = 0
    b_goals = 0
    outcome = 0

    random_event = np.random.random()
    #print(random_event)
    counter = 0


    for index, row in pd.DataFrame(data=game).iterrows():
        for x in range(len(row)):
            counter += row[x]
            
            a_goals = index
            b_goals = x
            
            if counter > random_event:
                break
        if counter > random_event:
            break
    
    if verbose == True:
        #print('Random float: {}'.format(random_event))
        print('{} goals: {}'.format(a_team, str(a_goals)))
        print('{} goals: {}\n'.format(b_team, str(b_goals)))
    
    return a_goals, b_goals

def show_game_summary(game, team_pred, a_team='A Team', b_team='B Team', verbose=False):
    a_team_odds = np.sum(np.tril(game, -1))
    b_team_odds = np.sum(np.triu(game, 1))
    draw_odds = np.sum(np.diag(game))
    
    if verbose == True:
        print('{} chance of Win: {}%'.format(a_team, round(a_team_odds*100, 2)))
        print('{} chance of Win: {}%'.format(b_team, round(b_team_odds*100, 2)))
        print('Odds of a draw: {}%'.format(round(draw_odds*100, 2)))
    
    a_goals, b_goals = simulate_goals(team_pred, game, a_team, b_team, verbose=verbose)
    #print(a_goals, b_goals)
    return a_team_odds, b_team_odds, draw_odds, a_goals, b_goals

# predicao dos gols dos times em uma partida
def team_preds(foot_model, a_team, b_team, max_goals=5, verbose=False):
    a_team_stats = get_team_stats(a_team)
    b_team_stats = get_team_stats(b_team)

    data1 = {'att': a_team_stats[0],
              'defe': a_team_stats[1],
              'away_att': b_team_stats[0], 
              'away_def': b_team_stats[1]
              }
    #print(data1)
    
    data2 = {'att': b_team_stats[0],
              'defe': b_team_stats[1],
              'away_att': a_team_stats[0], 
              'away_def': a_team_stats[1]
            }
    #print(data2)
        
    a_goals_avg = foot_model.predict(pd.DataFrame(data=data1, index=[1])).values[0]
    b_goals_avg = foot_model.predict(pd.DataFrame(data=data2, index=[1])).values[0]
    #print(a_goals_avg)
    #print(b_goals_avg)
    
    lambda_a = a_team_stats[0] / b_team_stats[1]
    lambda_b = b_team_stats[0] / a_team_stats[1]
    
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [lambda_a, lambda_b]]
    #print(max(team_pred[0]))
    #print(team_pred[1])
    #game = (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
    #print(game)
    #show_game_summary(game, a_team, b_team, verbose=verbose)
    return team_pred

# simulacao de uma partida, retornando a matriz que nos interessa
def simulate_match(team_pred, foot_model, a_team, b_team, max_goals=5, verbose=False):
    game = (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
    show_game_summary(game, team_pred, a_team, b_team, verbose=verbose)
    return game

In [22]:
team_predz = team_preds(poisson_model, 'Iran', 'Brazil')
game = simulate_match(team_predz, poisson_model, 'Iran', 'Brazil')
#pd.DataFrame(game)
show_game_summary(game, team_predz, 'Iran', 'Brazil', verbose=True)

Iran chance of Win: 29.42%
Brazil chance of Win: 24.12%
Odds of a draw: 13.52%
Iran goals: 4
Brazil goals: 2



(0.29421749211191345, 0.24123603085072506, 0.135238786313197, 4, 2)

In [19]:
pd.DataFrame(game)

,0,1,2,3,4,5
0,0.020229,0.041607,0.042788,0.029336,0.015084,0.006205
1,0.037299,0.076716,0.078895,0.054090,0.027813,0.011441
2,0.034387,0.070727,0.072735,0.049867,0.025642,0.010548
3,0.021135,0.043470,0.044704,0.030649,0.015760,0.006483
4,0.009742,0.020038,0.020607,0.014128,0.007265,0.002988
5,0.003593,0.007389,0.007599,0.005210,0.002679,0.001102


In [16]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math

def get_numbers_from_array(array):
    nums = []
    for n in array:
        try:
            nums.append(n[0])
        except:
            nums.append(n)
    return nums

# funcao para calcular as metricas do model
# tenho utilizado como base a metrica r2, que e um valor entre 0 e 1, sendo 1 o modelo perfeito
# o maximo que consegui chegar foi 0.21 e alguma coisa, depois de variar os parametros
# o que ainda e muito baixo, mas nao consegui melhorar muito mais infelizmente
# talvez se nossa predicao fosse de vitoria/derrota poderiamos ter um resultado melhor
def model_metrics(dataframe, poisson_model):
    yreal = []
    ypred = []
    team_pred = []
    i = 0
    yreal = list(dataframe['goals'])
    Y = pd.DataFrame(dataframe['goals'])
    X = dataframe.drop(columns=['goals', 'team', 'opponent'])
    res = sm.Poisson(Y.astype(float), X).fit()
    ypred = poisson_model.predict()
    ypred = get_numbers_from_array(ypred)
    print(ypred)
    #for index, row in dataframe.iterrows():
    #    team_pred = team_preds(poisson_model, row['team'], row['opponent'])
        #print(team_pred)
    #    yreal.append(row['goals'])
    #    ypred.append(team_pred[0].index(max(team_pred[0])))
    #print(yreal)
    #print(ypred)
    rms = math.sqrt(mean_squared_error(yreal, ypred))
    r2 = r2_score(yreal, ypred)
    return rms, r2

metrics = model_metrics(goal_model_data, poisson_model)
print(metrics)

Optimization terminated successfully.
         Current function value: 1.358956
         Iterations 9
[1.8719132408492123, 1.3547011335967254, 2.3648701911375856, 1.570754724467021, 2.031019335880602, 1.5945078735486247, 3.8215274962664503, 1.0355607232231432, 1.386604913317394, 2.371407975006289, 2.4848085941381117, 0.5542476602944536, 1.962472588055902, 0.7202228650968037, 0.8921331231774753, 1.102786812807405, 0.6342108715946903, 0.6937552054339048, 1.4754649422267447, 0.4603292110192469, 1.390249381611464, 0.6310503222523461, 0.6342108715946903, 0.6528975285457499, 0.8364617955928845, 1.829764904569395, 2.153351610465343, 1.9077459361187312, 1.7022850379236474, 0.6250831988062898, 1.8719132408492123, 2.080489566188281, 0.8300553160594212, 1.145880374371087, 1.5832396852018027, 1.893368410327737, 1.5487506109505005, 1.4754649422267447, 0.678016375904811, 1.1155511018987048, 1.3410025185047638, 1.5793423190889802, 1.0488777874735882, 0.31894117352880563, 0.8830083218377631, 1.5935145

In [17]:
def normalize_test_model(y):
    if y > (14/3):
        return 5
    elif y > (11/3):
        return 4
    elif y > (8/3):
        return 3
    elif y > (4/3):
        return 2
    elif y < (2/3):
        return 0
    else:
        return 1
    
def compare_accuracy(df):
    count1 = 0

    for index, row in df.iterrows():
        if row['predicted'] == row['goals']:
            count1 += 1
        else:
            continue

    print('Poisson Model: {}% accurate'.format((count1 / len(test_model_df))*100))

#print(goal_model_data)
X = goal_model_data.drop(columns=['goals', 'team', 'opponent'])
Y = pd.DataFrame(goal_model_data['goals'])
#print(X)
#print(np.asarray(X))
res = sm.Poisson(Y.astype(float), X).fit()
poisson_y_pred = poisson_model.predict()
#print(poisson_y_pred)
poisson_y_pred = get_numbers_from_array(poisson_y_pred)
print(poisson_y_pred)
ml_data = [poisson_y_pred]

test_model_df = pd.DataFrame(ml_data, 
                             index=['predicted']).transpose()

test_model_df['goals'] = list(goal_model_data['goals'])
test_model_df['predicted'] = test_model_df['predicted'].apply(normalize_test_model)
print(test_model_df)
compare_accuracy(test_model_df)
#goal_model_data.tail()

Optimization terminated successfully.
         Current function value: 1.358956
         Iterations 9
[1.8719132408492123, 1.3547011335967254, 2.3648701911375856, 1.570754724467021, 2.031019335880602, 1.5945078735486247, 3.8215274962664503, 1.0355607232231432, 1.386604913317394, 2.371407975006289, 2.4848085941381117, 0.5542476602944536, 1.962472588055902, 0.7202228650968037, 0.8921331231774753, 1.102786812807405, 0.6342108715946903, 0.6937552054339048, 1.4754649422267447, 0.4603292110192469, 1.390249381611464, 0.6310503222523461, 0.6342108715946903, 0.6528975285457499, 0.8364617955928845, 1.829764904569395, 2.153351610465343, 1.9077459361187312, 1.7022850379236474, 0.6250831988062898, 1.8719132408492123, 2.080489566188281, 0.8300553160594212, 1.145880374371087, 1.5832396852018027, 1.893368410327737, 1.5487506109505005, 1.4754649422267447, 0.678016375904811, 1.1155511018987048, 1.3410025185047638, 1.5793423190889802, 1.0488777874735882, 0.31894117352880563, 0.8830083218377631, 1.5935145

      predicted  goals
0             2      1
1             2      2
2             2      4
3             2      1
4             2      1
5             2      0
6             4      2
7             1      0
8             2      1
9             2      4
10            2      4
11            0      0
12            2      5
13            1      1
14            1      0
15            1      2
16            0      1
17            1      0
18            2      0
19            0      0
20            2      1
21            0      3
22            0      1
23            0      1
24            1      1
25            2      1
26            2      2
27            2      2
28            2      2
29            0      1
...         ...    ...
2560          2      0
2561          2      1
2562          1      1
2563          1      1
2564          2      1
2565          2      2
2566          1      1
2567          1      0
2568          1      2
2569          1      0
2570          1      2
2571       